# Authorization and Imports


In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%load_ext google.colab.data_table

In [3]:
%%bigquery --project plucky-zodiac-283501 df
SELECT *
FROM cleandata.combineddata

In [4]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [5]:
import pandas as pd

In [7]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tokenizing").getOrCreate()

# Read in Data and Place in Dataframe

In [8]:
# Read in data from storage
from pyspark import SparkFiles

url ="https://storage.googleapis.com/ucb_finalproject_nn/final/1975_1989.csv"
url2 = "https://storage.googleapis.com/ucb_finalproject_nn/final/1990_2002.csv"
url3 = "https://storage.googleapis.com/ucb_finalproject_nn/final/2003_2010.csv"
url4 = "https://storage.googleapis.com/ucb_finalproject_nn/final/2011_2016.csv"


In [9]:
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("1975_1989.csv"), sep=",", header=True)

In [10]:
spark.sparkContext.addFile(url2)
df2 = spark.read.csv(SparkFiles.get("1990_2002.csv"), sep=",", header=True)

In [11]:
spark.sparkContext.addFile(url3)
df3 = spark.read.csv(SparkFiles.get("2003_2010.csv"), sep=",", header=True)

In [12]:
spark.sparkContext.addFile(url4)
df4 = spark.read.csv(SparkFiles.get("2011_2016.csv"), sep=",", header=True)

In [13]:
dfUnion = df.union(df2)

In [14]:
dfUnion2 = dfUnion.union(df3)

In [15]:
dfUnion3 = dfUnion.union(df4)

In [16]:
dfUnion3.show()

+----+-----+-------+--------------------+------+-------+------------------+-------+
|year|state|station|                name|   lat|   elev|          avg_temp|avg_co2|
+----+-----+-------+--------------------+------+-------+------------------+-------+
|1989|   AK| 700260|W POST-WILL ROGER...|71.283|+0009.5|13.548002432155659| 352.91|
|1989|   AK| 700635|GALBRAITH LAKE AI...|68.483|+0814.0|14.455962301587302| 352.91|
|1989|   AK| 700636|         PRUDHOE BAY| 70.25|+0014.0|14.731031746031746| 352.91|
|1989|   AK| 700637|   DEADHORSE AIRPORT|70.192|+0018.6|14.480375704045056| 352.91|
|1989|   AK| 701040|CAPE LISBURNE LRR...|68.883|+0004.9|21.721833814333813| 352.91|
|1989|   AK| 701170|TIN CITY LRRS AIR...|65.567|+0083.0|23.785535714285714| 352.91|
|1989|   AK| 701190|   PORT CLARENCE CGS|65.254|+0003.0|26.277756285875974| 352.91|
|1989|   AK| 701195|SHISHMAREF/NEW AI...| 66.25|+0004.0|24.586995890495611| 352.91|
|1989|   AK| 701330|RALPH WIEN MEMORI...|66.867|+0009.1|23.547707373271887| 

# Machine Learning: Train, Test, Fit and Summarize

In [17]:
from pyspark.ml.feature import VectorAssembler

In [21]:
df_modified = dfUnion3.drop('name', 'state', 'station',)

# station = six-digit number that is a string, no decimals. 

df_modified.show()

+----+------+-------+------------------+-------+
|year|   lat|   elev|          avg_temp|avg_co2|
+----+------+-------+------------------+-------+
|1989|71.283|+0009.5|13.548002432155659| 352.91|
|1989|68.483|+0814.0|14.455962301587302| 352.91|
|1989| 70.25|+0014.0|14.731031746031746| 352.91|
|1989|70.192|+0018.6|14.480375704045056| 352.91|
|1989|68.883|+0004.9|21.721833814333813| 352.91|
|1989|65.567|+0083.0|23.785535714285714| 352.91|
|1989|65.254|+0003.0|26.277756285875974| 352.91|
|1989| 66.25|+0004.0|24.586995890495611| 352.91|
|1989|66.867|+0009.1|23.547707373271887| 352.91|
|1989|69.367|+0085.0|15.843559552255787| 352.91|
|1989|  67.1|+0102.0|22.108897978763569| 352.91|
|1989|  66.0|+0389.0|   25.264856000481| 352.91|
|1989|66.918|+0193.2|  22.7935291858679| 352.91|
|1989|66.817|+0335.0|18.404166666666669| 352.91|
|1989|65.175|+0067.7|25.802313748079875| 352.91|
|1989|65.933|+0155.0| 13.62361111111111| 352.91|
|1989|66.567|+0135.6|22.343321812596002| 352.91|
|1989|64.511|+0004.0

In [22]:
df_modified = df_modified.selectExpr("cast(year as int) year",
    "cast(lat as double) lat",
    "cast(elev as double) elev", 
    "cast(avg_temp as double) avg_temp", 
    "cast(avg_co2 as double) avg_co2")

df_modified.printSchema()

root
 |-- year: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- elev: double (nullable = true)
 |-- avg_temp: double (nullable = true)
 |-- avg_co2: double (nullable = true)



In [23]:
features = df_modified.drop("avg_temp")

In [24]:
assembler = VectorAssembler(inputCols=features.columns, outputCol="features", handleInvalid="skip")

In [25]:
output = assembler.transform(df_modified).select('features','avg_temp')

In [26]:
train,test = output.randomSplit([0.75, 0.25], seed=42) 

In [27]:
train.show()

+--------------------+-----------------+
|            features|         avg_temp|
+--------------------+-----------------+
|[1975.0,18.255,10...|79.11708141321044|
|[1975.0,18.336,6....|81.09575010118418|
|[1975.0,19.719,11...|72.35902585765488|
|[1975.0,20.79,399...|71.07767868014932|
|[1975.0,20.9,15.5...|74.15759408602152|
|[1975.0,21.317,15...|75.18506464413724|
|[1975.0,21.487,25...|72.82809042675284|
|[1975.0,21.984,30...|75.04279185867894|
|[1975.0,22.033,7....|75.93634641896564|
|[1975.0,24.583,1....|78.99674859190989|
|[1975.0,25.483,2....|75.74278673835124|
|[1975.0,25.788,8....|76.77397593445983|
|[1975.0,25.914,7....|73.02472862263184|
|[1975.0,26.184,30...|73.21689132104456|
|[1975.0,27.5,15.2...|70.66938300051203|
|[1975.0,27.65,20....| 75.9071886140213|
|[1975.0,27.683,5....| 73.7376849718382|
|[1975.0,27.774,13...| 71.9824980798771|
|[1975.0,27.85,4.3...|73.53958781362006|
|[1975.0,27.911,3....| 75.3717601126472|
+--------------------+-----------------+
only showing top

In [28]:
test.show()

+--------------------+-----------------+
|            features|         avg_temp|
+--------------------+-----------------+
|[1975.0,18.433,2....|79.68193484383002|
|[1975.0,21.155,13...|76.06389411853509|
|[1975.0,21.324,2....|75.23113799283155|
|[1975.0,21.45,7.3...|75.23592165898617|
|[1975.0,24.557,0....|79.18689260112649|
|[1975.0,27.401,9....|74.34494815668202|
|[1975.0,27.741,54...| 70.8595801331285|
|[1975.0,28.233,3....|76.31795285359802|
|[1975.0,28.367,20...|68.96269393241167|
|[1975.0,28.457,14...|70.37797555043521|
|[1975.0,28.725,3....|69.42279889912955|
|[1975.0,28.78,17....|76.25212877624169|
|[1975.0,29.383,21...|67.82131784434205|
|[1975.0,29.617,10...|68.55820148489504|
|[1975.0,29.638,14...|69.04866807475679|
|[1975.0,29.692,50...|69.67728814644138|
|[1975.0,29.817,0....|68.32069252432157|
|[1975.0,29.98,29....|67.26517729134665|
|[1975.0,30.2,165....|67.01954813108037|
|[1975.0,30.371,14...|56.60704990897196|
+--------------------+-----------------+
only showing top

In [29]:
from pyspark.ml.regression import LinearRegression
lin_reg = LinearRegression(featuresCol = 'features', labelCol='avg_temp')
linear_model = lin_reg.fit(train)
linear_model2 = lin_reg.fit(test)

In [30]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("\nr2: %f" % trainSummary.r2)

RMSE: 4.379378

r2: 0.856924


Root MSE: absolute measure of fit. Standard deviation of unexplained variance / square root of the variance of the residuals. 

R-squared: Relative measure of fit. Approximately three-quarters of the observed variation can be explained by the model's inputs. 

In [31]:
testSummary = linear_model2.summary
print("RMSE: %f" % testSummary.rootMeanSquaredError)
print("\nr2: %f" % testSummary.r2) # 74.5202% 

RMSE: 4.286972

r2: 0.863175


Root MSE: absolute measure of fit. standard deviation of unexplained variance / square root of the variance of the residuals. 

R-squared: relative measure of fit. approximately three-quarters of the observed variation can be explained by the model's inputs.

# Make Predictions Using Trained Model


In [32]:
url7 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/US_Capitols_new.csv"

In [33]:
spark.sparkContext.addFile(url7)
df7 = spark.read.csv(SparkFiles.get("US_Capitols_new.csv"), sep=",", header=True)

In [34]:
df8 = df7[["Year","State", "City"]]

df8.show()

+----+-----+------------+
|Year|State|        City|
+----+-----+------------+
|2025|   AL|  Montgomery|
|2025|   AK|      Juneau|
|2025|   AZ|     Phoenix|
|2025|   AR| Little Rock|
|2025|   CA|  Sacramento|
|2025|   CO|      Denver|
|2025|   CT|    Hartford|
|2025|   DE|       Dover|
|2025|   FL| Tallahassee|
|2025|   GA|     Atlanta|
|2025|   HI|    Honolulu|
|2025|   ID|       Boise|
|2025|   IL| Springfield|
|2025|   IN|Indianapolis|
|2025|   IA|  Des Moines|
|2025|   KS|      Topeka|
|2025|   KY|   Frankfort|
|2025|   LA| Baton Rouge|
|2025|   ME|     Augusta|
|2025|   MD|   Annapolis|
+----+-----+------------+
only showing top 20 rows



In [35]:
df7 = df7.drop('State', 'City')

In [36]:
df_modified2 = df7.drop('Elevation_ft', 'Dew_Point')

df_modified2.show()

+----+-------+-----------+------+
|Year|    Lat|Elevation_m|   CO2|
+----+-------+-----------+------+
|2025|   32.3|     73.152|424.49|
|2025|  58.37|    17.0688|424.49|
|2025|  33.43|   331.0128|424.49|
|2025|  35.22|    102.108|424.49|
|2025|  38.52|      9.144|424.49|
|2025|  39.75|   1609.344|424.49|
|2025|  41.73|    17.9832|424.49|
|2025|  39.13|      9.144|424.49|
|2025|  30.38|    61.8744|424.49|
|2025|  33.65|     320.04|424.49|
|2025|21.3294|   6.001512|424.49|
|2025|43.6007|    832.104|424.49|
|2025|  39.85|   170.0784|424.49|
|2025|39.7771|    217.932|424.49|
|2025|  41.53|    291.084|424.49|
|2025|  39.07|    288.036|424.49|
|2025|38.1924|   155.1432|424.49|
|2025|  30.53|    17.0688|424.49|
|2025|  44.32|    20.7264|424.49|
|2025|  38.97|    11.8872|424.49|
+----+-------+-----------+------+
only showing top 20 rows



In [37]:
df_modified2 = df_modified2.selectExpr("cast(Year as int) Year",
    "cast(Lat as double) Lat",
    "cast(Elevation_m as double) Elevation_m", 
    "cast(CO2 as double) CO2")

In [38]:
features2 = df_modified2

features2.show()

+----+-------+-----------+------+
|Year|    Lat|Elevation_m|   CO2|
+----+-------+-----------+------+
|2025|   32.3|     73.152|424.49|
|2025|  58.37|    17.0688|424.49|
|2025|  33.43|   331.0128|424.49|
|2025|  35.22|    102.108|424.49|
|2025|  38.52|      9.144|424.49|
|2025|  39.75|   1609.344|424.49|
|2025|  41.73|    17.9832|424.49|
|2025|  39.13|      9.144|424.49|
|2025|  30.38|    61.8744|424.49|
|2025|  33.65|     320.04|424.49|
|2025|21.3294|   6.001512|424.49|
|2025|43.6007|    832.104|424.49|
|2025|  39.85|   170.0784|424.49|
|2025|39.7771|    217.932|424.49|
|2025|  41.53|    291.084|424.49|
|2025|  39.07|    288.036|424.49|
|2025|38.1924|   155.1432|424.49|
|2025|  30.53|    17.0688|424.49|
|2025|  44.32|    20.7264|424.49|
|2025|  38.97|    11.8872|424.49|
+----+-------+-----------+------+
only showing top 20 rows



In [39]:
assembler2 = VectorAssembler(inputCols=features2.columns, outputCol="features", handleInvalid="skip")

In [40]:
output2 = assembler2.transform(features2).select('features')

output2.show()

+--------------------+
|            features|
+--------------------+
|[2025.0,32.3,73.1...|
|[2025.0,58.37,17....|
|[2025.0,33.43,331...|
|[2025.0,35.22,102...|
|[2025.0,38.52,9.1...|
|[2025.0,39.75,160...|
|[2025.0,41.73,17....|
|[2025.0,39.13,9.1...|
|[2025.0,30.38,61....|
|[2025.0,33.65,320...|
|[2025.0,21.3294,6...|
|[2025.0,43.6007,8...|
|[2025.0,39.85,170...|
|[2025.0,39.7771,2...|
|[2025.0,41.53,291...|
|[2025.0,39.07,288...|
|[2025.0,38.1924,1...|
|[2025.0,30.53,17....|
|[2025.0,44.32,20....|
|[2025.0,38.97,11....|
+--------------------+
only showing top 20 rows



In [41]:
lin_reg2 = LinearRegression(featuresCol = 'features')

In [42]:
lr_predictions = linear_model.transform(output2)

In [43]:
lr_predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[2025.0,32.3,73.1...| 65.57672383755704|
|[2025.0,58.37,17....|34.595073530901715|
|[2025.0,33.43,331...| 63.05412428059477|
|[2025.0,35.22,102...|61.946883271139505|
|[2025.0,38.52,9.1...|58.414315647343216|
|[2025.0,39.75,160...|49.688095768829626|
|[2025.0,41.73,17....| 54.52818507741475|
|[2025.0,39.13,9.1...|57.683442687305856|
|[2025.0,30.38,61....| 67.92828923690857|
|[2025.0,33.65,320...|62.840262130717655|
|[2025.0,21.3294,6...| 79.02551669682484|
|[2025.0,43.6007,8...|  48.5970095296376|
|[2025.0,39.85,170...|56.091379439141534|
|[2025.0,39.7771,2...| 55.96184068915805|
|[2025.0,41.53,291...| 53.53005684729354|
|[2025.0,39.07,288...|56.491326006616845|
|[2025.0,38.1924,1...| 58.14512677600224|
|[2025.0,30.53,17....|  67.9516361660491|
|[2025.0,44.32,20....| 51.41253753406633|
|[2025.0,38.97,11....| 57.86271422609303|
+--------------------+------------

In [44]:
lr_pandas = lr_predictions.toPandas()

lr_pandas.head()

,features,prediction
0,"[2025.0, 32.3, 73.152, 424.49]",65.576724
1,"[2025.0, 58.37, 17.0688, 424.49]",34.595074
2,"[2025.0, 33.43, 331.0128, 424.49]",63.054124
3,"[2025.0, 35.22, 102.108, 424.49]",61.946883
4,"[2025.0, 38.52, 9.144, 424.49]",58.414316


In [45]:
df8_pandas = df8.toPandas()

df8_pandas.head()

,Year,State,City
0,2025,AL,Montgomery
1,2025,AK,Juneau
2,2025,AZ,Phoenix
3,2025,AR,Little Rock
4,2025,CA,Sacramento


In [46]:
combined_df = pd.concat([df8_pandas, lr_pandas], axis=1)

In [47]:
combined_df.head()

,Year,State,City,features,prediction
0,2025,AL,Montgomery,"[2025.0, 32.3, 73.152, 424.49]",65.576724
1,2025,AK,Juneau,"[2025.0, 58.37, 17.0688, 424.49]",34.595074
2,2025,AZ,Phoenix,"[2025.0, 33.43, 331.0128, 424.49]",63.054124
3,2025,AR,Little Rock,"[2025.0, 35.22, 102.108, 424.49]",61.946883
4,2025,CA,Sacramento,"[2025.0, 38.52, 9.144, 424.49]",58.414316


In [48]:
final_predictions = combined_df.drop(['features'], axis=1)

In [49]:
final_predictions.head()

,Year,State,City,prediction
0,2025,AL,Montgomery,65.576724
1,2025,AK,Juneau,34.595074
2,2025,AZ,Phoenix,63.054124
3,2025,AR,Little Rock,61.946883
4,2025,CA,Sacramento,58.414316


In [50]:
final_predictions.to_csv('capital_predictions.csv', index=False) 